In [49]:
import tensorflow as tf
import math

In [50]:
def logit(r1, r2, theta1, theta2):
    return r1*r2*np.cos(theta1 - theta2)

In [51]:
def grad_u(r1, r2, theta1, theta2):
    return -1.0*r1*r2*np.sin(theta1 - theta2)

In [52]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [53]:
def grad(theta1, theta2):
    error = sigmoid(logit(r1, r2, theta1, theta2))-1
    return error*grad_u(r1, r2, theta1, theta2)

In [54]:
def tf_logits(r1,r2,theta1,theta2):
    cos = tf.cos(theta1-theta2)
    rad = tf.multiply(r1,r2)
    return tf.multiply(cos, rad)

In [188]:
example = 1
theta1 = tf.Variable(math.pi/3.0, name='theta1')
theta2 = tf.Variable(math.pi/6.0, name='theta2')
theta3 = tf.Variable(math.pi/3.0, name='theta1')
theta4 = tf.Variable(math.pi/6.0, name='theta2')
radius1 = tf.Variable(1.0)
radius2 = tf.Variable(1.0)
radius3 = tf.Variable(1.0)
radius4 = tf.Variable(1.0)

b = tf.Variable(1.0, name='b')
true_logits = tf_logits(radius1,radius2,theta1,theta2)
sampled_logits = tf_logits(radius3,radius4,theta3,theta4)
true_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=true_logits, labels=tf.ones_like(true_logits))
sampled_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=sampled_logits, labels=tf.zeros_like(sampled_logits))
loss = true_xent + sampled_xent + b
opt = tf.train.GradientDescentOptimizer(1.0)
grads = opt.compute_gradients(loss, [theta1, theta2, theta3, theta4])
apply_grad = opt.apply_gradients(grads)

In [189]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(10):
        b_val = 1.0    
        t1, t2 = sess.run([theta1, theta2])
        theta1_grad = grad(t1, t2)
        print('grad should be: ', theta1_grad)
        grad_vals = sess.run(grads, feed_dict={b: b_val})
        print(grad_vals)
        sess.run(apply_grad, feed_dict={b: b_val})
#         assert(grad_vals[0][0] == theta1_grad)

# r = tf.Variable([2.0,1.0, 3.0], name='r')
# # b placeholder (simualtes the "data" part of the training)
# b = tf.placeholder(tf.float32)
# # make model (1/2)(x-b)^2
# example_radius = tf.nn.embedding_lookup(r, example)
# example_theta = tf.nn.embedding_lookup(theta, example)
# loss = example_radius*tf.cos(example_theta) - b
# learning_rate = 1.0


TypeError: 'list' object is not callable

In [179]:
x = tf.Variable([0,1,2,3], dtype=tf.float32)
y = tf.Variable([0,1,2,3], dtype=tf.float32)
ex1 = tf.placeholder(tf.int32)
ex2 = tf.placeholder(tf.int32)
# example = tf.Variable(1)
a = tf.nn.embedding_lookup(x,ex1)
b = tf.nn.embedding_lookup(y,ex2)
c = tf.cos(tf.subtract(a, b))
loss = c - 1.0
opt = tf.train.GradientDescentOptimizer(1.0)
grad_y = opt.compute_gradients(loss, [y])
grad_x = opt.compute_gradients(loss, [x])
apply_grad = opt.apply_gradients(grad_y + grad_x)

In [180]:
def grad_y_fun(y, yidx, x, xidx):
    print('evaluating grad at y: ', y, 'and index: ', yidx)
    yvals = y[yidx]
    xvals = x[xidx]
#     print('vals are: ', vals)
    return np.sin(xvals - yvals)

In [181]:
def check_equivalence(autograd, grad):
    autos = autograd[0][0].values
    for x,y in zip(grad, autos):
        assert(x==y)

In [187]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(20):
        e1 = np.array([1, 1])
        e2 = np.array([2, 3])
        gy, gx, y_val, x_val = sess.run([grad_y, grad_x, y, x], feed_dict={ex1:e1, ex2:e2})
#         print(cn)
        print(y_val)
        print(gy)
        my_grad = grad_y_fun(y_val, e2, x_val, e1)
        check_equivalence(gy, my_grad)
        print('grad should be: ', grad_y_fun(y_val, e2, x_val, e1))
        _, cn, yval = sess.run([apply_grad, c, y], feed_dict={ex1:e1, ex2:e2})
        print(yval)
        print(cn)  
        

[ 0.  1.  2.  3.]
[(IndexedSlicesValue(values=array([-0.84147096, -0.90929741], dtype=float32), indices=array([2, 3], dtype=int32), dense_shape=array([4], dtype=int32)), array([ 0.,  1.,  2.,  3.], dtype=float32))]
('evaluating grad at y: ', array([ 0.,  1.,  2.,  3.], dtype=float32), 'and index: ', array([2, 3]))
('evaluating grad at y: ', array([ 0.,  1.,  2.,  3.], dtype=float32), 'and index: ', array([2, 3]))
('grad should be: ', array([-0.84147096, -0.90929741], dtype=float32))
[ 0.          1.          2.84147096  3.90929747]
[ 0.54030228 -0.41614684]
[ 0.          1.          2.84147096  3.90929747]
[(IndexedSlicesValue(values=array([ 0.4355478 ,  0.99863142], dtype=float32), indices=array([2, 3], dtype=int32), dense_shape=array([4], dtype=int32)), array([ 0.        ,  1.        ,  2.84147096,  3.90929747], dtype=float32))]
('evaluating grad at y: ', array([ 0.        ,  1.        ,  2.84147096,  3.90929747], dtype=float32), 'and index: ', array([2, 3]))
('evaluating grad at y: 

In [200]:
-0.00614413/4

-0.0015360325

In [201]:
-0.01318374/4

-0.003295935

In [203]:
x = np.array([1,2,3])
y = np.where(x==2)
x[y]

array([2])